In [1]:
import numpy as np
from journal import *

In [10]:
# Given data
diameter = 100e-3  # Diameter of the bearing in meters
length = 30e-3     # Length of the bearing in meters
load = 525         # Static load in Newtons
clearance = 0.1e-3  # Radial clearance in meters
viscosity = 0.1    # Viscosity of the oil in Pa.s
speed_rpm = 1500   # Rotational speed in revolutions per minute

# Convert speed to rad/s
speed_rad_s = (2 * np.pi * speed_rpm) / 60


In [11]:
eps, Ss, S = compute_eps(diameter, length, load, clearance, viscosity,speed_rpm)

print("epsilon: ", eps)

epsilon:  0.26629750900547655


In [13]:
# Friswell equations
Ke, Ce = compute_ke_ce(eps, load / clearance, Omega=speed_rpm * 2 * np.pi / 60)
print("Stiffness matrix (Ke) [MN/m]:")
print(Ke / 1e6)  # Convert to MN/m

print("\nDamping matrix (Ce) [kNs/m]:")
print(Ce / 1e3)  # Convert to kNs/m

Stiffness matrix (Ke) [MN/m]:
[[ 12.80795958  16.39359325]
 [-25.06039328   8.81530274]]

Damping matrix (Ce) [kNs/m]:
[[232.89692987 -81.92437052]
 [-81.92437052 294.91161944]]


In [27]:
# Al-Bender equations
nu = speed_rad_s
pa = 1010325
Lambda = np.array([compute_squeeze_number(viscosity,nu, pa, diameter/2, clearance)])
sigma = np.array([compute_squeeze_number(viscosity,nu,pa,diameter/2,clearance)])

LoverD = length / diameter

Kxx, Kxy = stiffness_Kxx_Kxy(Lambda,sigma,LoverD)
Cxx, Cxy = damping_Cxx_Cxy(Lambda,sigma,LoverD)

Kxx = Kxx[0]
Kxy = Kxy[0]
Cxx = Cxx[0]
Cxy = Cxy[0]

K_scale = pa * length * diameter / clearance
C_scale = 3 * viscosity * length / (clearance / diameter)**3

Ke = np.array([
    [Kxx,Kxy],
    [-Kxy,Kxx]
]) * K_scale

Ce = np.array([
    [Cxx,Cxy],
    [-Cxy,Cxx]
]) * C_scale

print("Stiffness matrix (Ke) [MN/m]:")
print(Ke / 1e6)  # Convert to MN/m

print("\nDamping matrix (Ce) [kNs/m]:")
print(Ce / 1e3)  # Convert to kNs/m

Stiffness matrix (Ke) [MN/m]:
[[17.94013968 -6.23659052]
 [ 6.23659052 17.94013968]]

Damping matrix (Ce) [kNs/m]:
[[  39.70336834 -114.21047641]
 [ 114.21047641   39.70336834]]
